## Understanding problem statement

- Objective: Create a ML model that optimizes the channel distribution by maximizing market reach and revenue

### Phase 1: EDA
- Understand how variables differ across channels and region
- Visualize on week, city and channel levels
- Insights: Action items to optmize distribution channel

## Ingesting the data

In [ ]:
# READING ALL THE DATA IN PANDAS DATAFRAME

import pandas as pd
city_df = pd.read_csv('../../data/raw/city.csv', encoding='ISO-8859-1')
competitors_df = pd.read_csv('../../data/raw/competitivelandscape.csv')
consumer_behaviour_df = pd.read_csv('../../data/raw/consumerbehavior.csv')
market_influencers_df = pd.read_csv('../../data/raw/externalmarketinfluencers.csv')
products_df = pd.read_csv('../../data/raw/products.csv')
sales_df = pd.read_csv('../../data/raw/retailsalesdistribution.csv')

## Univariate 

### Sales data

In [ ]:
sales_df.head()

In [ ]:
from ydata_profiling import ProfileReport

sales_data_profile = ProfileReport(sales_df, title='Sales Data Profiling Report', explorative=True)


In [ ]:
sales_data_profile.to_file('../../eda/sales_data_profile.html')

- 20 cities, 7 SKUs, 5 channels

### Competitive Behavior

In [ ]:
from ydata_profiling import ProfileReport

competitors_data_profile = ProfileReport(competitors_df, title='competitors_Data Profiling Report', explorative=True)

competitors_data_profile.to_file('../../eda/competitors_data_profile.html')


### Products 

In [ ]:
from ydata_profiling import ProfileReport

products_data_profile = ProfileReport(products_df, title='Products Data Profiling Report', explorative=True)

products_data_profile.to_file('../../eda/products_data_profile.html')


### External Market Influence

In [ ]:
from ydata_profiling import ProfileReport

external_market_influence_data_profile = ProfileReport(market_influencers_df, title='External Market Influence Data Profiling Report', explorative=True)

external_market_influence_data_profile.to_file('../../eda/external_market_influence_data_profile.html')


### Customer Behavior 

In [ ]:
from ydata_profiling import ProfileReport

customer_behavior_data_profile = ProfileReport(consumer_behaviour_df, title='Customer Behavior Data Profiling Report', explorative=True)

customer_behavior_data_profile.to_file('../../eda/customer_behavior_data_profile.html')


### City

In [ ]:
from ydata_profiling import ProfileReport

city_data_profile = ProfileReport(city_df, title='City Data Profiling Report', explorative=True)

city_data_profile.to_file('../../eda/city_data_profile.html')


# Data Preprocessing

### Sales Data

In [ ]:

# STANDARDIZING THE DATE FORMAT IN SALES DATAFRAME
import pandas as pd
sales_df_processed = sales_df.copy()

sales_df_processed['Date'] = pd.to_datetime(sales_df_processed['Date'], format='%Y-%m-%d')

sales_df_processed['Date'] = pd.to_datetime(sales_df_processed['Date']).dt.strftime('%Y-%m-%d')

sales_df_processed.head()



In [ ]:
# save data
sales_df_processed.to_csv('../../data/processed/sales_processed.csv', index=False)

### Competitor Data

In [ ]:

# STANDARDIZING THE DATE FORMAT IN SALES DATAFRAME
import pandas as pd
competitors_df_processed = competitors_df.copy()

competitors_df_processed['Date'] = pd.to_datetime(competitors_df_processed['Date'], format='%Y-%m-%d')

competitors_df_processed['Date'] = pd.to_datetime(competitors_df_processed['Date']).dt.strftime('%Y-%m-%d')

competitors_df_processed.head()



In [ ]:
# save data
competitors_df_processed.to_csv('../../data/processed/competitors_processed.csv', index=False)

### Products

In [ ]:
import pandas as pd
import re
import numpy as np

products_df_processed = products_df.copy()

# Convert "Flavor Variant" column to categorical data type
products_df_processed['Flavor Variant'] = products_df_processed['Flavor Variant'].astype('category')


# 2. Convert Launch Date from MM/DD/YYYY to yyyy-mm-dd
products_df_processed['Launch Date'] = pd.to_datetime(products_df_processed['Launch Date (MM/DD/YYYY)'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

def convert_pack_size(pack_str):
    """
    Extract numeric value and convert pack size to litres.
    Assumes the pack size string contains a number and a unit (ml or L).
    """
    match = re.search(r'([\d\.]+)\s*(ml|l)', pack_str, re.IGNORECASE)
    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()
        if unit == 'ml':
            return value / 1000  # convert millilitres to litres
        else:
            return value
    return np.nan

products_df_processed['Pack Size (L)'] = products_df_processed['Pack Size (ml/L)'].apply(convert_pack_size)

channels = {
    'General Trade': r'general trade',
    'E Commerce': r'e[-\s]?commerce',
    'Modern Trade': r'modern trade',
    'HoReCa': r'horeca',
    'Q Commerce': r'q[-\s]?commerce'
}

def encode_distribution(dist_str):
    """
    Returns a Series with one-hot encoding for each distribution channel.
    """
    dist_lower = dist_str.lower()
    result = {}
    for channel, pattern in channels.items():
        result[channel] = 1 if re.search(pattern, dist_lower) else 0
    return pd.Series(result)

dist_df = products_df_processed['Distribution Coverage'].apply(encode_distribution)
products_df_processed = pd.concat([products_df_processed, dist_df], axis=1)  

products_df_processed = products_df_processed[['Product Name', 'Flavor Variant', 'SKU Identification Number', 
          'Launch Date', 'Pack Size (L)', 'General Trade', 'E Commerce', 
          'Modern Trade', 'HoReCa', 'Q Commerce']]

products_df_processed.head(100)






In [ ]:
# save data
products_df_processed.to_csv('../../data/processed/products_processed.csv', index=False)

### External Market influence

In [ ]:
import pandas as pd

market_influencers_df_processed = market_influencers_df.copy()

# convert data to yyyy-mm-dd format
market_influencers_df_processed['Week_Start_Date'] = pd.to_datetime(market_influencers_df_processed['Week_Start_Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

market_influencers_df_processed['Festival'] = market_influencers_df_processed['Festival'].fillna("No Festival")


market_influencers_df_processed.head()


In [ ]:
# save data
market_influencers_df_processed.to_csv('../../data/processed/market_influencers_processed.csv', index=False)

### Customer Behavior

In [ ]:

import pandas as pd

consumer_behaviour_df_processed = consumer_behaviour_df.copy()
consumer_behaviour_df_processed['Purchase_Frequency'] = consumer_behaviour_df_processed['Purchase_Frequency'].fillna(consumer_behaviour_df_processed['Purchase_Frequency'].median())

consumer_behaviour_df_processed['Preferred_Channel'] = consumer_behaviour_df_processed['Preferred_Channel'].replace('HoReca', 'HoReCa')

consumer_behaviour_df_processed['Active'] = consumer_behaviour_df_processed['Active'].map({True: 1, False: 0})



consumer_behaviour_df_processed.head()


In [ ]:
# save data
consumer_behaviour_df_processed.to_csv('../../data/processed/consumer_behavior_processed.csv', index=False)

### City

In [ ]:

import pandas as pd

city_df_processed = city_df.copy()

city_df_processed = city_df_processed.dropna(axis=1, how='all')
city_df_processed['City_Name'] = city_df_processed['City_Name'].astype('category')

city_df_processed['Per_Capita_Income (INR)'] = city_df_processed['Per_Capita_Income (INR)'].astype(str).str.replace(',', '')
city_df_processed['Per_Capita_Income (INR)'] = pd.to_numeric(city_df_processed['Per_Capita_Income (INR)'], errors='coerce')

city_df_processed.head()



In [ ]:
# save data
city_df_processed.to_csv('../../data/processed/city_processed.csv', index=False)

## Generating reports of processed data

In [ ]:
# READ CITY, COMPETITOR, CONSUMER BEHAVIOR, MARKET INFLUENCE, PRODUCTS AND SALES DATA FROM PROCESSED FOLDER AND CREATE Y DATA PROFILING REPORTS AND SAVE THEM IN EDA/PROCESSED FOLDER
import pandas as pd
from ydata_profiling import ProfileReport

city_df = pd.read_csv('../../data/processed/city_processed.csv')
competitors_df = pd.read_csv('../../data/processed/competitors_processed.csv')
consumer_behaviour_df = pd.read_csv('../../data/processed/consumer_behavior_processed.csv')
market_influencers_df = pd.read_csv('../../data/processed/market_influencers_processed.csv')
products_df = pd.read_csv('../../data/processed/products_processed.csv')
sales_df = pd.read_csv('../../data/processed/sales_processed.csv')


sales_data_profile = ProfileReport(sales_df, title='Sales Data Profiling Report', explorative=True)

sales_data_profile.to_file('../../eda/reports/processed/sales_data_profile.html')

competitors_data_profile = ProfileReport(competitors_df, title='competitors_Data Profiling Report', explorative=True)

competitors_data_profile.to_file('../../eda/reports/processed/competitors_data_profile.html')

products_data_profile = ProfileReport(products_df, title='Products Data Profiling Report', explorative=True)

products_data_profile.to_file('../../eda/reports/processed/products_data_profile.html')

external_market_influence_data_profile = ProfileReport(market_influencers_df, title='External Market Influence Data Profiling Report', explorative=True)

external_market_influence_data_profile.to_file('../../eda/reports/processed/external_market_influence_data_profile.html')

customer_behavior_data_profile = ProfileReport(consumer_behaviour_df, title='Customer Behavior Data Profiling Report', explorative=True)

customer_behavior_data_profile.to_file('../../eda/reports/processed/customer_behavior_data_profile.html')

city_data_profile = ProfileReport(city_df, title='City Data Profiling Report', explorative=True)

city_data_profile.to_file('../../eda/reports/processed/city_data_profile.html')



# EDA

In [1]:
import pandas as pd
sales_df = pd.read_csv('../../data/processed/sales_processed.csv')
sales_df.head()

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0


In [11]:
import plotly.express as px

# Sales by SKU (Revenue)
sales_sku = sales_df.groupby('SKU_ID', as_index=False)['Sales'].sum()
fig = px.bar(sales_sku, x='SKU_ID', y='Sales', title='Sales Revenue by SKU')
fig.show()

# Sales by Channel
sales_channel = sales_df.groupby('Channel', as_index=False)['Sales'].sum()
fig = px.pie(sales_channel, names='Channel', values='Sales', title='Sales Distribution by Channel')
fig.show()

# Sales by Channel
sales_channel = sales_df.groupby('Channel', as_index=False)['Units_Sold'].sum()
fig = px.pie(sales_channel, names='Channel', values='Units_Sold', title='Units_Sold Distribution by Channel')
fig.show()


# Sales by City
# Sales by City
sales_city = sales_df.groupby('City_ID', as_index=False)['Sales'].sum()
fig = px.pie(sales_city, names='City_ID', values='Sales', title='Sales Revenue by City')
fig.show()

# Sales by Date
sales_date = sales_df.groupby('Date', as_index=False).agg({'Sales': 'sum', 'Units_Sold': 'sum'})
fig = px.line(sales_date, x='Date', y=['Sales', 'Units_Sold'], title='Sales Revenue and Units Sold by Date')
fig.show()


In [1]:
import pandas as pd
import plotly.express as px

# Load datasets
city = pd.read_csv('../../data/processed/city_processed.csv')
competitor = pd.read_csv('../../data/processed/competitors_processed.csv')
consumer = pd.read_csv('../../data/processed/consumer_behavior_processed.csv')
external = pd.read_csv('../../data/processed/market_influencers_processed.csv')
sales = pd.read_csv('../../data/processed/sales_processed.csv')
products = pd.read_csv('../../data/processed/products_processed.csv')

# Create Selling_Price = Sales / Units_Sold
sales['Selling_Price'] = sales['Sales'] / sales['Units_Sold']

# Convert dates to datetime in all datasets
sales['Date'] = pd.to_datetime(sales['Date'])
external['Week_Start_Date'] = pd.to_datetime(external['Week_Start_Date'])
competitor['Date'] = pd.to_datetime(competitor['Date'])



# Merge with city data
merged = sales.merge(city, on='City_ID', how='left')

merged = merged.merge(
    products, 
    left_on='SKU_ID', 
    right_on='SKU Identification Number', 
    how='left'
)

merged = merged.merge(
    consumer, 
    on="City_ID",
    how='left'
)



# # Add week start date to sales data
# merged['Week_Start_Date'] = merged['Date'] - pd.to_timedelta(merged['Date'].dt.dayofweek, unit='D')

# # Merge with external market data (now aligned on datetime)
# merged = merged.merge(
#     external, 
#     left_on=['Week_Start_Date', 'City_ID'], 
#     right_on=['Week_Start_Date', 'City_ID'], 
#     how='left'
# )

# # Merge with competitor data (filter for Minute Maid)
# competitor_mm = competitor[competitor['Brand'] == 'Minute Maid']
# merged = merged.merge(
#     competitor_mm, 
#     left_on=['Week_Start_Date', 'Channel'], 
#     right_on=['Date', 'Channel'], 
#     suffixes=('', '_competitor'), 
#     how='left'
# )

# # Merge consumer behavior (aggregate by city and channel)
# # Standardize channel names to match (e.g., "E-commerce" vs "E Commerce")
# consumer['Preferred_Channel'] = consumer['Preferred_Channel'].str.replace('-', ' ', regex=False)
# merged['Channel'] = merged['Channel'].str.replace('-', ' ', regex=False)

# consumer_agg = consumer.groupby(['City_ID', 'Preferred_Channel']).agg(
#     Avg_Income=('Income_Level', 'mean'),
#     Preferred_Flavor=('Preferred_Flavor', lambda x: x.mode()[0]),
#     Active_Customers=('Active', 'sum')
# ).reset_index()

# merged = merged.merge(
#     consumer_agg, 
#     left_on=['City_ID', 'Channel'], 
#     right_on=['City_ID', 'Preferred_Channel'], 
#     how='left'
# )

merged.head(1000)

MemoryError: Unable to allocate 6.23 GiB for an array with shape (835750000,) and data type int64

In [20]:
merged.to_csv('../../data/aggregated/merged_data_daily_granularity.csv', index=False)

In [32]:
# drop City_ID, SKU_ID, Product_Name, SKU Identification Name, Launch Date
merged = pd.read_csv('../../data/aggregated/merged_data_daily_granularity.csv')
merged = merged.drop(columns=['City_ID', 'SKU_ID', 'Product Name', 'SKU Identification Number', 'Launch Date', "Channel", "Selling_Price"])
merged.head(1000)

,Date,Units_Sold,Sales,City_Name,City_tier,Population_Density(persons/km),Per_Capita_Income (INR),Flavor Variant,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce
0,2023-01-01,268,80.4,Delhi,Tier 1,14893,461910,Mixed Fruit,1.00,1,1,1,1,1
1,2023-01-01,168,50.4,Delhi,Tier 1,14893,461910,Orange,1.00,1,1,1,1,1
2,2023-01-01,521,156.3,Delhi,Tier 1,14893,461910,Apple,1.00,1,1,0,0,0
3,2023-01-01,247,74.1,Delhi,Tier 1,14893,461910,Mixed Fruit,1.00,1,1,1,1,1
4,2023-01-01,161,12075.0,Delhi,Tier 1,14893,461910,Orange,0.25,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,803,60225.0,Pune,Tier 1,6034,336503,Orange,0.25,1,1,1,1,0
996,2023-01-03,209,62.7,Pune,Tier 1,6034,336503,Orange,1.00,1,1,1,1,1
997,2023-01-03,167,12525.0,Pune,Tier 1,6034,336503,Mixed Fruit,0.25,1,1,1,1,0
998,2023-01-03,2366,709.8,Pune,Tier 1,6034,336503,Apple,1.00,1,1,0,0,0


In [33]:
pd.set_option('display.max_columns', None)

# One-hot encode categorical columns
merged = pd.get_dummies(merged, columns=['City_Name', 'City_tier', 'Flavor Variant', 'Pack Size (L)'], dtype=int)
merged.head(1000)

,Date,Units_Sold,Sales,Population_Density(persons/km),Per_Capita_Income (INR),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name_Ahmedabad,City_Name_Bengaluru,City_Name_Bhopal,City_Name_Chandigarh,City_Name_Chennai,City_Name_Delhi,City_Name_Guwahati,City_Name_Hyderabad,City_Name_Jaipur,City_Name_Jammu,City_Name_Kochi,City_Name_Kolkata,City_Name_Lucknow,City_Name_Ludhiana,City_Name_Mumbai,City_Name_Nagpur,City_Name_Patna,City_Name_Pune,City_Name_Surat,City_Name_Visakhapatnam,City_tier_Tier 1,City_tier_Tier 2,Flavor Variant_Apple,Flavor Variant_Guava,Flavor Variant_Mixed Fruit,Flavor Variant_Orange,Pack Size (L)_0.135,Pack Size (L)_0.25,Pack Size (L)_1.0
0,2023-01-01,268,80.4,14893,461910,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,2023-01-01,168,50.4,14893,461910,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
2,2023-01-01,521,156.3,14893,461910,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
3,2023-01-01,247,74.1,14893,461910,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
4,2023-01-01,161,12075.0,14893,461910,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,803,60225.0,6034,336503,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
996,2023-01-03,209,62.7,6034,336503,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
997,2023-01-03,167,12525.0,6034,336503,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
998,2023-01-03,2366,709.8,6034,336503,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1


In [34]:
merged.to_csv('../../data/aggregated/merged_data_daily_granularity_ohe.csv', index=False)